# Edit Your Expectation Suite
Use this notebook to recreate and modify your expectation suite:

**Expectation Suite Name**: `primeiros_passos_great-expectation-suite-taxi.demo`


In [ ]:
import datetime

import pandas as pd

import great_expectations as ge
import great_expectations.jupyter_ux
from great_expectations.core.batch import BatchRequest
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.exceptions import DataContextError

context = ge.data_context.DataContext()

# Note that if you modify this batch request, you may save the new version as a .json file
#  to pass in later via the --batch-request option
batch_request = {'datasource_name': 'primeiros_passos_great-expectations', 'data_connector_name': 'default_inferred_data_connector_name', 'data_asset_name': 'yellow_tripdata_sample_2019-01.csv', 'limit': 1000}


# Feel free to change the name of your suite here. Renaming this will not remove the other one.
expectation_suite_name = "primeiros_passos_great-expectation-suite-taxi.demo"
try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.')
except DataContextError:
    suite = context.create_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')


validator = context.get_validator(
    batch_request=BatchRequest(**batch_request),
    expectation_suite_name=expectation_suite_name
)
column_names = [f'"{column_name}"' for column_name in validator.columns()]
print(f"Columns: {', '.join(column_names)}.")
validator.head(n_rows=5, fetch_all=False)

## Create & Edit Expectations


Add expectations by calling specific expectation methods on the `validator` object. They all begin with `.expect_` which makes autocompleting easy using tab.

Because you selected interactive mode, you are now creating or editing an Expectation Suite with validator feedback from the sample batch of data that you specified (see `batch_request`).

Note that if you select manual mode you may still create or edit an Expectation Suite directly, without feedback from the `validator`. See our documentation for more info and examples: [How to create a new Expectation Suite without a sample batch](https://docs.greatexpectations.io/docs/guides/expectations/how_to_create_and_edit_expectations_based_on_domain_knowledge_without_inspecting_data_directly).



You can see all the available expectations in the **[expectation gallery](https://greatexpectations.io/expectations)**.

### Table Expectation(s)

In [ ]:

validator.expect_table_columns_to_match_ordered_list(column_list=['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge'])

In [ ]:

validator.expect_table_row_count_to_be_between(max_value=10000, min_value=10000)

### Column Expectation(s)

#### `passenger_count`

In [ ]:

validator.expect_column_min_to_be_between(column='passenger_count', max_value=1, min_value=1)

In [ ]:

validator.expect_column_max_to_be_between(column='passenger_count', max_value=6, min_value=6)

In [ ]:

validator.expect_column_mean_to_be_between(column='passenger_count', max_value=1.5716, min_value=1.5716)

In [ ]:

validator.expect_column_median_to_be_between(column='passenger_count', max_value=1.0, min_value=1.0)

In [ ]:

validator.expect_column_quantile_values_to_be_between(column='passenger_count', allow_relative_error='lower', quantile_ranges={'quantiles': [0.05, 0.25, 0.5, 0.75, 0.95], 'value_ranges': [[1, 1], [1, 1], [1, 1], [2, 2], [5, 5]]})

In [ ]:

validator.expect_column_values_to_be_in_set(column='passenger_count', value_set=[1, 2, 3, 4, 5, 6])

In [ ]:

validator.expect_column_values_to_not_be_null(column='passenger_count')

In [ ]:

validator.expect_column_proportion_of_unique_values_to_be_between(column='passenger_count', max_value=0.0006, min_value=0.0006)

In [ ]:

validator.expect_column_values_to_be_in_type_list(column='passenger_count', type_list=['INTEGER', 'integer', 'int', 'int_', 'int8', 'int16', 'int32', 'int64', 'uint8', 'uint16', 'uint32', 'uint64', 'Int8Dtype', 'Int16Dtype', 'Int32Dtype', 'Int64Dtype', 'UInt8Dtype', 'UInt16Dtype', 'UInt32Dtype', 'UInt64Dtype', 'INT', 'INTEGER', 'INT64', 'TINYINT', 'BYTEINT', 'SMALLINT', 'BIGINT', 'IntegerType', 'LongType'])

## Review & Save Your Expectations

Let's save the expectation suite as a JSON file in the `great_expectations/expectations` directory of your project.

Let's now rebuild your Data Docs, which helps you communicate about your data with both machines and humans.

In [ ]:
print(validator.get_expectation_suite(discard_failed_expectations=False))
validator.save_expectation_suite(discard_failed_expectations=False)

checkpoint_config = {
    "class_name": "SimpleCheckpoint",
    "validations": [
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ]
}
checkpoint = SimpleCheckpoint(
    f"{validator.active_batch_definition.data_asset_name}_{expectation_suite_name}",
    context,
    **checkpoint_config
)
checkpoint_result = checkpoint.run()

context.build_data_docs()

validation_result_identifier = checkpoint_result.list_validation_result_identifiers()[0]
context.open_data_docs(resource_identifier=validation_result_identifier)